In [ ]:
import os
import sys
sys.path.append("/home/ubuntu/gradient")
from core.config.bria4B_adapt import HyperParemeter as model_config
from core.config.dataset import DatasetConfig
from models.diffusion.bria4B_adapt.train import Bria3_0 as Model

bria_conf = model_config(
    train_batch_size=2,
    precompute=False,
    dense_caption_ratio = 0.8,  # 1
    gradient_accumulation_steps = 1,
    flow_matching_latent_loss=1,
    shift=3.0,
    use_flow_matching=1,
    use_dynamic_shift=1,
    weighting_scheme="uniform"
    )

model = Model(bria_conf)

In [7]:
## initialize the strategy
from core.config.startegy import FSDPConfig

fsdp_config = FSDPConfig(
        strategy_name="FSDP",
        devices=4,
        mixed_precision=True,
        sharding_strategy="FULL_SHARD",
        auto_wrap_policy="transformer_auto_wrap_policy",
    )

In [3]:
#Data loader
from core.config.dataloader import DataLoaderConfig

dataset = DatasetConfig(
    dataset_name="timm/imagenet-1k-wds",
    train_batch_size=2,
)

In [8]:
from core.config.trainer import TrainerConfig
from core.config.loggers import WandB
import datetime

time = datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")

logger = WandB(
    wandb_mode="offline",
    wandb_project="EA-tests",
    wandb_group="test-1",
    wandb_run_name=f"test-{time}"
)

# Trainer 
trainer = TrainerConfig(
    max_train_steps=100
    
)

In [ ]:
# Train

model.train(
    dataset_config=dataset,
    trainer_config=trainer,
    logger_config=logger,
)